In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-23 20:20:49--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-23 20:20:49 (35.8 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)   

In [10]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [21]:
from openai import OpenAI

In [22]:
client = OpenAI()

In [14]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [16]:
response.choices[0].message.content

"It's possible that you can still enroll in a course even if it has already started, but this depends on several factors, including the policies of the institution offering the course and the timing of your request. Here are some steps you can take:\n\n1. **Check the Institution's Policy:** Review the enrollment policies on the institution’s website or in the course catalog. Some institutions allow late enrollment within a certain period after the course begins.\n\n2. **Contact the Instructor:** Reach out to the course instructor directly. Explain your situation and express your interest in the course. The instructor may have the discretion to allow you to join late or provide you with makeup materials.\n\n3. **Consult the Admissions Office:** Get in touch with the admissions or registrar's office. They can provide specific details about deadlines and exceptions for late enrollment.\n\n4. **Catch Up on Coursework:** If you are allowed to enroll late, be prepared to catch up quickly. Re

In [24]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't containt the answer, output NONE.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [32]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [33]:
prompt = prompt_template.format(question=q, context=context).strip()

In [34]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use the facts from the CONTEXT when answer answering the QUESTION.
If the CONTEXT doesn't containt the answer, output NONE.

QUESTION: the course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on you

In [35]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects. So, don't leave everything for the last minute."

In [1]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )   

    return results

In [15]:
def build_prompt(query, search_results):

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't containt the answer, output NONE.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [19]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [31]:
query = 'How do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [32]:
rag(query)

'To run Kafka, you can follow these steps based on the provided context for running Java Kafka applications:\n\n1. Navigate to your project directory.\n2. Run the Java producer/consumer/kstreams/etc. in the terminal using the following command:\n\n```sh\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the actual name of your jar file.\n\nFor Python Kafka, ensure you create a virtual environment and install necessary dependencies as specified:\n\n```sh\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nsource env/bin/activate # Run this every time you need the virtual env\n```\n\nFor Windows, activating the virtual environment is slightly different:\n\n```sh\nenv\\Scripts\\activate\n```\n\nTo deactivate the virtual environment, use:\n\n```sh\ndeactivate\n```\n\nMake sure the Docker images are up and running as well.\n\nFor permissions issues, you can run:\n\n```sh\nchmod +x b

In [33]:
from elasticsearch import Elasticsearch

In [34]:
es_client = Elasticsearch('http://localhost:9200')

In [36]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [37]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 41.47it/s]


In [47]:
query = 'I just discovered the course. Can I still join it?'

In [58]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [54]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [55]:
rag(query)

"Yes, you can still join the course even if you just discovered it. You don't need to register and are eligible to submit the homeworks. However, keep in mind that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."

In [59]:
def elastic_search_homework(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])

    return result_docs

In [61]:
elastic_search_homework('How do I execute a command in a running docker container?')

[75.54128, 43.922554, 38.684105, 38.33403, 35.94081]